In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import glob
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')
mpl.rcParams['savefig.dpi'] = 128
mpl.rcParams['figure.dpi'] = 128
# Plot size to 14" x 7"
mpl.rc('figure', figsize = (14, 7))
# Font size to 14
mpl.rc('font', size = 14)
# Do not display top and right frame lines
mpl.rc('axes.spines', top = False, right = False)
# Remove grid lines
mpl.rc('axes', grid = False)
# Set backgound color to white
mpl.rc('axes', facecolor = 'white')

In [2]:
df = pd.read_csv("2009-2017.csv")

In [3]:
df.shape

(268639, 9)

In [4]:
print("Shape of data frame: {}".format(df.shape))
print("Keys of enquiries_dataset: \n{}".format(df.keys()))
print("data ytpes of enquiries_dataset: \n{}".format(df.dtypes))

Shape of data frame: (268639, 9)
Keys of enquiries_dataset: 
Index(['classification', 'loan amount', 'loan reason', 'property use',
       'enquiry status', 'month', 'day', 'hour', 'weekday'],
      dtype='object')
data ytpes of enquiries_dataset: 
classification    object
loan amount        int64
loan reason       object
property use      object
enquiry status    object
month              int64
day                int64
hour               int64
weekday            int64
dtype: object


In [5]:
test = df[df['enquiry status'] == 'Rejected']
test.shape

(218370, 9)

In [6]:
test = df[df['enquiry status'] == 'Accepted']
test.shape

(50269, 9)

In [7]:
from sklearn.utils import resample

df_majority = df[df['enquiry status'] == 'Rejected']
df_minority = df[df['enquiry status'] == 'Accepted']


# Upsample minority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,     # sample with replacement
                                 n_samples=50269,    # to match majority class
                                 random_state=123) # reproducible results



# Combine majority class with upsampled minority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# Display new class counts
df_downsampled['enquiry status'].value_counts()

Accepted    50269
Rejected    50269
Name: enquiry status, dtype: int64

In [8]:
df = df_downsampled

In [9]:
print("Shape of new data frame: {}".format(df.shape))

Shape of new data frame: (100538, 9)


In [10]:
print("data ytpes of enquiries_dataset: \n{}".format(df.dtypes))

data ytpes of enquiries_dataset: 
classification    object
loan amount        int64
loan reason       object
property use      object
enquiry status    object
month              int64
day                int64
hour               int64
weekday            int64
dtype: object


In [11]:
target = 'enquiry status';
# df = df.iloc[:number_of_rows]
X = df[df.keys()]
X = df.loc[:,df.columns != target]

y = df[target]

In [12]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

# X['classification'] = le.fit_transform(X['classification'])

In [13]:
category_column = X.select_dtypes(include='object')

In [14]:
# X = MultiColumnLabelEncoder(columns = category_column.columns).fit_transform(X)
# X = pd.get_dummies(X, columns=category_column.columns)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


class EncodeCategorical(BaseEstimator, TransformerMixin):
    """
    Encodes a specified list of columns or all columns if None.
    """

    def __init__(self, columns=None):
        self.columns  = [col for col in columns]
        self.encoders = None

    def fit(self, data, target=None):
        """
        Expects a data frame with named columns to encode.
        """
        # Encode all columns if columns is None
        if self.columns is None:
            self.columns = data.columns


        # Fit a label encoder for each column in the data frame
        self.encoders = {
            column: LabelEncoder().fit(data[column])
            for column in self.columns
        }
        return self

    def transform(self, data):
        """
        Uses the encoders to transform a data frame.
        """
        output = data.copy()
        for column, encoder in self.encoders.items():
            print(column)
            print(data[column])
            output[column] = encoder.transform(data[column])

        return output
    
    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
# ec = EncodeCategorical(columns=category_column.columns)
# ec.fit(X)
# ec.transform(X)
from sklearn.preprocessing import LabelEncoder

le_dict = {col: LabelEncoder() for col in category_column.columns }
for col in columns:
    X[col] = le_dict[col].fit_transform(X[col])

In [ ]:
category_mapping = dict(zip(category_column.columns, {}))

In [ ]:
for col in category_mapping : 
    category_mapping[col] = 

In [17]:
X.shape

(100538, 8)

In [18]:
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [19]:
to_drop

[]

In [20]:
from sklearn.model_selection import train_test_split

# split data and labels into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y,
                     test_size=0.4,
                     random_state=0,
                     stratify=y)

## Selecting Best Models Using Exhaustive Search

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model

# Create logistic regression
logistic = linear_model.LogisticRegression()

# Create range of candidate penalty hyperparameter values
penalty = ['l1', 'l2']

# Create range of candidate regularization hyperparamet values
C = np.logspace(0, 4, 10)

# Create dictionary hyperparameter candidates
hyperparameters = dict(C=C, penalty=penalty)

# Create grid search
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)

# Fit grid search
best_model = gridsearch.fit(X, y)

In [22]:
# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Penalty: l2
Best C: 1.0


## Creating A Baseline Classification Model

In [23]:
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Create dummy classifer
# dummy = DummyClassifier(strategy='uniform', random_state=1)
pipe_dummy = make_pipeline(StandardScaler(),
                       DummyClassifier(strategy='uniform', random_state=1))

# "Train" model
pipe_dummy.fit(X_train, y_train)

# Get accuracy score
pipe_dummy.score(X_test, y_test)

0.50096976327829723

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pipe_rf = make_pipeline(StandardScaler(),
                        RandomForestClassifier(n_estimators=1000,random_state=1))

pipe_rf.fit(X_train, y_train)
rf_label = pipe_rf.predict(X_test)
print('Test Accuracy: %.3f' % pipe_rf.score(X_test, y_test))

Test Accuracy: 0.745


In [25]:
# from sklearn.model_selection import learning_curve

# train_sizes, train_scores, test_scores = learning_curve(estimator=pipe_rf,
#                                                         X=X_train,y=y_train,train_sizes=np.linspace(0.1, 1.0, 10),
#                                                         cv=10,n_jobs=1)
# train_mean = np.mean(train_scores,axis=1)
# train_std = np.std(train_scores, axis=1)      
# test_mean = np.mean(test_scores, axis=1)
# test_std = np.std(test_scores, axis=1)


# plt.plot(train_sizes, train_mean,color='blue', marker='o',markersize=5,label='training accuracy')
# plt.fill_between(train_sizes,train_mean + train_std,train_mean - train_std,alpha=0.15, color='blue')
# plt.plot(train_sizes, test_mean,color='green', linestyle='--',marker='s', markersize=5,label='validation accuracy')
# plt.fill_between(train_sizes,test_mean + test_std,test_mean - test_std,alpha=0.15, color='green')
# plt.grid()
# plt.xlabel('Number of training samples')
# plt.ylabel('Accuracy')               
# plt.legend(loc='lower right')
# plt.ylim([0.8, 1.0])
# plt.show()

In [26]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn import cross_validation

scores = cross_validation.cross_val_score(pipe_rf, X, y, cv=5)
print("Random forest cross_validation: {:.2f}".format(np.mean(scores, axis=0)))

print("Random forest")
print(classification_report(y_test, rf_label))

/Users/mluo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Random forest cross_validation: 0.68
Random forest
             precision    recall  f1-score   support

          0       0.71      0.82      0.76     20108
          1       0.79      0.67      0.73     20108

avg / total       0.75      0.74      0.74     40216



In [27]:
prob_y_2 = pipe_rf.predict_proba(X)
prob_y_2 = [p[1] for p in prob_y_2]
print( roc_auc_score(y, prob_y_2) )

0.969317593181


In [28]:
print( np.unique( rf_label ) )

[0 1]


In [29]:
# from sklearn.ensemble import RandomForestClassifier

# from sklearn import metrics

# # rescale data
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# rfc = RandomForestClassifier(n_estimators=500,random_state=1)
# rfc.fit(X_train_scaled, y_train)
# pred_forest = rfc.predict(X_test)

# print("Test score: {:.2f}".format(rfc.score(X_test_scaled, y_test)))


In [ ]:
from sklearn.model_selection import learning_curve
train_sizes, train_scores, test_scores = learning_curve(estimator=pipe_rf,
                                                        X=X_train,y=y_train,train_sizes=np.linspace(0.1, 1.0, 10),
                                                        cv=10,n_jobs=1)
train_mean = np.mean(train_scores,axis=1)
train_std = np.std(train_scores, axis=1)      
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)


plt.plot(train_sizes, train_mean,color='blue', marker='o',markersize=5,label='training accuracy')
plt.fill_between(train_sizes,train_mean + train_std,train_mean - train_std,alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean,color='green', linestyle='--',marker='s', markersize=5,label='validation accuracy')
plt.fill_between(train_sizes,test_mean + test_std,test_mean - test_std,alpha=0.15, color='green')
plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')               
plt.legend(loc='lower right')
plt.ylim([0.8, 1.0])
plt.show()

In [ ]:
plt.savefig('myfig.png')

In [31]:
classifer = pipe_rf.steps[1]

In [32]:
feat_labels = X.columns[0:]
importances = pipe_rf.steps[1][1].feature_importances_

# reverse the list
indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,feat_labels[indices[f]],importances[indices[f]]))
    
# plt.title('Feature Importance')
# plt.bar(range(X_train.shape[1]),importances[indices],align='center')
# plt.xticks(range(X_train.shape[1]),feat_labels[indices], rotation=90)
# plt.xlim([-1, X_train.shape[1]])
# plt.tight_layout()
# plt.show()

 1) loan amount                    0.360307
 2) day                            0.140194
 3) hour                           0.127441
 4) classification                 0.111714
 5) month                          0.095004
 6) weekday                        0.066136
 7) loan reason                    0.065642
 8) property use                   0.033563


In [33]:
# classifer.oob_score_

In [34]:
# import pickle
# import os

# dest = os.path.join('movieclassifier', 'pkl_objects')
# if not os.path.exists(dest):
#     os.makedirs(dest)

# pickle.dump(stop,open(os.path.join(dest, 'stopwords.pkl'),'wb'),protocol=4)
# pickle.dump(clf,
            
# from sklearn.externals import joblib
# joblib.dump(pipe_rf, 'app/models/classifier.pkl')

In [35]:
from sklearn.externals import joblib
joblib.dump(pipe_rf, 'app/models/classifier.pkl')

['app/models/classifier.pkl']

In [36]:
model_columns = list(X.columns)
joblib.dump(model_columns, 'app/models/model_columns.pkl')

['app/models/model_columns.pkl']

In [105]:
joblib.dump(ec, 'app/models/encoder.pkl')

['app/models/encoder.pkl']

In [106]:
clf = joblib.load('app/models/classifier.pkl')
model_columns = joblib.load('app/models/model_columns.pkl')

In [119]:
import json
my_json_string = json.dumps({
    "classification": "REA",
    "marketing_code": "IPH",
    "enquired":"29/5/15 17:10",
    "loan amount": "210000",
    "property_use": "Residence",
    "loan_reason": "Refinance"
})
my_json_string

'{"classification": "REA", "marketing_code": "IPH", "enquired": "29/5/15 17:10", "loan amount": "210000", "property_use": "Residence", "loan_reason": "Refinance"}'

In [120]:
# test = pd.read_json(my_json_string, orient='index')
# data = pd.read_json(my_json_string, typ='series',orient='index')
# data = pd.DataFrame(data=data)
data = json.loads(my_json_string)
data = pd.DataFrame(data,index=[0])

In [121]:
def transform_cols(df):
    df.columns = map(str.lower, df.columns)
    df.columns = df.columns.str.replace('_', ' ')
    return df

In [122]:
transform_cols(data)

,classification,enquired,loan amount,loan reason,marketing code,property use
0,REA,29/5/15 17:10,210000,Refinance,IPH,Residence


In [123]:
def transform(df): 
    df['loan amount'] = df['loan amount'].astype('float')
    df['enquired'] = pd.DatetimeIndex(df['enquired'])
    df['month'] = df['enquired'].dt.month
    df['day'] = df['enquired'].dt.day
    df['hour'] = df['enquired'].dt.hour
    df['weekday'] = df['enquired'].dt.dayofweek
    
    if 'post code' in df.columns: 
        df['post code'] = df['post code'].astype('int')
    
    if 'enquired'in df.columns:
        df.drop(['enquired'], axis = 1, inplace = True)

In [124]:
transform(data)

In [125]:
data

,classification,loan amount,loan reason,marketing code,property use,month,day,hour,weekday
0,REA,210000.0,Refinance,IPH,Residence,5,29,17,4


In [126]:
data = ec.transform(data)

0    REA
Name: classification, dtype: object


TypeError: '<' not supported between instances of 'int' and 'str'

In [81]:
data = pd.get_dummies(data)

data

,loan amount,month,day,hour,weekday,classification_REA,loan reason_Refinance,marketing code_IPH,property use_Residence
0,210000.0,5,29,17,4,1,1,1,1


In [61]:
def cleanFeatures(data) :
    for col in model_columns: 
        if col not in data.columns:
            data[col] = 0

#  test any error column in test data
for col in data.columns: 
    if col not in model_columns:
        data.drop(col, axis = 1, inplace = True)

In [62]:
cleanFeatures(data)

In [ ]:
prediction = clf.predict(data)

In [ ]:
prediction

In [ ]:
print("posibility is: {}".format(np.max(clf.predict_proba(data))))

In [ ]:
data = test.loc[:,test.columns != target]

In [ ]:
data['classification'] = le.transform(data['classification'])

In [ ]:
data = pd.get_dummies(data)

In [ ]:
cleanFeatures(data)

In [ ]:
data.shape

In [ ]:
model_columns

In [ ]:
prediction = clf.predict(data)

In [ ]:
prediction

In [ ]:
proba = []
result = clf.predict_proba(data)
for row in range(len(result)):
    proba.append(np.max(result[row]))

In [ ]:
data = test.loc[:,test.columns != target]

In [ ]:
data["prediction"] = prediction
data["re"] = proba

In [ ]:
data[data['prediction'] == 1]